# Midrex ND Plant 

In this example we will calculate the heat requirement considering midrex reformer and shaft furnace in a Midrex Natural Gas Plant by utilizing stream objects.

Stream calculation in ChemApp for Python allows setting initial conditions in addition to general boundary conditions. This enables quick calculation of heat balances and modeling scenarios.

![Midrex ND Plant flowchart ](MidrexND.png)


Source: MIDREX NG Brochure (4.12.2018)

## Reformer and Shaft Furnace in Midrex Reactor

The MIDREX Reformer plays a crucial role in the production of direct reduced iron (DRI) in the MIDREX Shaft Furnace. In the Shaft Furnace, iron oxide reacts with a hot gas containing H2 and CO, resulting in the reduction of iron oxide into metallic iron (Fe), with CO2 and H2O as byproducts. The MIDREX Reformer heats and catalytically reforms natural gas, recycling CO2 and H2O from the reduction process to generate a rich reducing gas for DRI production. The Reformer also recycles byproducts and unreacted gases to minimize energy consumption and provide the necessary energy for reduction reactions in the Shaft Furnace.

-> Assumptions have been made to streamline the model, taking into account only a subset of the components of the Midrex Furnace due to the complex nature of the process.

-> In this model, the process gas extracted from the top gas is not recycled into the reformer. Instead, a fixed gas composition is introduced into the reformer. However, the energy contribution from the shaft furnace, referred to as "Top Gas Fuel," has been incorporated into the energy balance equations.

## Initialization and general Setup
Import the Pandas and ChemApp for Python.

In [ ]:
import pandas as pd
from chemapp.friendly import (
    ThermochemicalSystem as cats,
    StreamCalculation as casc, 
    Units as cau,
    TemperatureUnit,AmountUnit,EnergyUnit,PressureUnit
)

## Load data file 
Determine the data file path. Open, read and close the thermochemical data-file midrex-database.cst (system Fe-O-H-C-N).

In [ ]:
#Load database 
cats.load("midrex_database.cst")

## Info about the CST file thermodynamic system

In [ ]:
print(cats.get_str_phs_pcs())

## Units setup

In [ ]:
#Set units 
cau.set_A_unit(AmountUnit.kg)
cau.set_T_unit(TemperatureUnit.C)
cau.set_P_unit(PressureUnit.bar)
cau.set_E_unit(EnergyUnit.kWh)

## Set Input Stream 
Routine to define ChemApp for Python stream and incoming amounts. 

In [ ]:
def set_input_stream(input_stream_dict: dict, reset_IA = False):
    #Manage incoming amounts
    if reset_IA == True:
        casc.remove_eq_conditions_IA()

    #Create stream object 
    casc.create_st(name = input_stream_dict["name"], T = input_stream_dict["T"], P = input_stream_dict["P"]) # C bar
    #Set incoming amount of the stream object
    for phpc in input_stream_dict["percentage"]:
            ph,pc = phpc
            casc.set_IA_pc(name = input_stream_dict["name"], ph =  ph , pc = pc, value = input_stream_dict["amount"] * (input_stream_dict["percentage"][phpc]/100)) #kg
    return

## Create equilibrium reaction calculator for the reformer 

In a simplified reformer`Stream Reformer` is heated up to an elevated `temperature`. An initial composition of a gas mixture `reformer_initial_input` is provided to reformer. As output `reducing_gas_stream` is obtained to be fed into the `Shaft Furnace`.

In previous code block, "Stream Reformer `reformer_initial_input` temperature is set to 580°C. Below, we will provide a reaction temperature that will also be equal to the output `reducing_gas_stream` temperature.

![Reformer flowchart ](reformer_chart.png)


In [ ]:
def reformer(input_stream: dict, reformer_T: float,reformer_P: float, print = False):
    #Remove equilibrium conditions
    casc.remove_eq_conditions_all()

    #Set input stream 
    set_input_stream(input_stream)

    #Set reaction temperature
    casc.set_eq_P(reformer_P)
    casc.set_eq_T(reformer_T)

    #Calculate equilibrium
    casc.calculate_eq(print_results= print)

    results = casc.get_result_object()
    
    #get amount of gases
    amount_pcs_in_gas = casc.get_eq_A_pcs_in_ph(0)
    initial_amount_pcs_in_gas = casc.get_eq_IA_pcs_in_ph(0)
    amount_total = casc.get_eq_A_ph(0)
    initial_amount_total = sum(casc.get_eq_IA_pcs_in_ph(0)) 
    #Get name of gasses for indexing
    name_pcs_in_gas = cats.get_name_pcs_in_ph(0)
    
    #get wt.% of gases 
    wt_pcs_in_gas = [gas_constituent / amount_total * 100 for gas_constituent in amount_pcs_in_gas]
    initial_wt_pcs_in_gas = [gas_constituent / initial_amount_total * 100 for gas_constituent in initial_amount_pcs_in_gas]

    #reformer output is the reducing gas stream
    reducing_gas_stream = results.create_stream(name='Stream Reducing Gas',include_zeros=False)
    #Create dataframe to have an idea about the process
    data = {
        "Gas Input kg": initial_amount_pcs_in_gas,
        "Gas Output kg": amount_pcs_in_gas,
        "Gas Input wt.%": initial_wt_pcs_in_gas,
        "Gas Output wt.%": wt_pcs_in_gas

    }

    df = pd.DataFrame(data, index=name_pcs_in_gas)

    #Filter dataframe 
    threshold = 1e-4 
    df_filtered =  df[(df["Gas Input kg"] > threshold) | (df["Gas Output kg"]> threshold)]
    enthalpy_change = results.dH 
    return reducing_gas_stream,df_filtered, enthalpy_change

## Create equilibrium reaction calculator for the shaft furnace

In a simplified scenerio Shaft furnace is operated at a certain temperature. `reducing_gas_stream` that obtained from Reformer is provided to the Shaft Furnace as well as the hematite ore is fed to the furnace at ambient conditions (25°C,1 bar) by `iron_oxide_stream`. Moreover, to reduce the CO2 emissions, `ammonia_stream` and `hydrogen_stream` is fed to the furnace in a desired ratio. As output `stream_top_gas` and `stream_DRI` is obtained.1/3 of the  gas stream will be used for energy conversation, whereas the 2/3 will be utilized in `Reformer`. The latter solid srteam contains the direct reduced iron, with a certain metallization degree.   

![Shaft furnace flowchart ](shaft_furnace_chart.png)


In [ ]:
def shaft_furnace(ammonia_stream:dict,hydrogen_stream: dict, iron_oxide_stream:dict, reducing_gas_stream, Furnace_T, Furnace_P, print = False): 
    #Remove equilibrium conditions
    casc.remove_eq_conditions_all()    
    
    #Set streams
    set_input_stream(ammonia_stream)
    set_input_stream(hydrogen_stream)
    set_input_stream(iron_oxide_stream)
    casc.set_st(reducing_gas_stream)

    #Set furnace T
    casc.set_eq_T(Furnace_T)
    casc.set_eq_P(Furnace_P)
    
    #Calculate and Print results if requested
    casc.calculate_eq(print_results=print)

    results = casc.get_result_object()

    #Define output streams 
    stream_top_gas = results.create_gas_stream("Stream Top-gas", include_zeros= False)
    stream_DRI = results.create_solid_stream("Stream DRI", include_zeros= False)
    enthalpy_change = results.dH


    #Calculate metallization degree
    #Assuming FCC is the forming phase for Fe
    total_amount_of_DRI_stream = stream_DRI.A
    amount_of_iron = results.phs["FCC_A1#1"].A
    metallization_degree = amount_of_iron/total_amount_of_DRI_stream
    return  stream_top_gas, stream_DRI, enthalpy_change, metallization_degree


## Energy calculator 
Calculate the energy required to supply considering the recovered heat from off-gas, and consumed energy based on reaction dH of reformer and shaft furnace.

In [ ]:
def energy_balance(top_gas_stream,shaft_furnace_dH,reformer_dH):
    #One third of the energy is recovered from topgas
    #Assuming 100% energy conversion for top_gas_fuel_enthalpy
    casc.set_st(top_gas_stream)
    #Cool down top gas to ambient temperature
    casc.set_eq_T(580)
    casc.calculate_eq(print_results= False)
    resuts = casc.get_result_object()
    #Get recovered enthalpy while cooling. 1/3 of the top gas will be reused as "fuel" for reformer.
    top_gas_fuel_enthalpy =  resuts.dH / 3
    shaft_furnace_enthalpy_change = shaft_furnace_dH
    reformer_enthalpy_change = reformer_dH
    total = reformer_enthalpy_change + shaft_furnace_enthalpy_change + top_gas_fuel_enthalpy
    return total ,top_gas_fuel_enthalpy


## Set the Midrex process 
Here we can provide input arguments and deploy functions to simulate the simplified midrex reactor. 

In [ ]:

#Reformer input 
# Define reformer input stream properties
initial_reformer_stream = {"name": "Reducing gas stream","T":580,"P":1,"amount":470, "percentage":{(0,"H2"): 3.92, (0,"CO"): 29.6, (0,"CO2"): 36.72, (0,"H2O"): 13.02, (0,"CH4"): 15.17, (0,"N2"): 1.56}} #C bar kg wt.%

# Run reformer()
reducing_gas_stream,df_reformer, reformer_enthalpy_change = reformer(initial_reformer_stream,reformer_T= 950, reformer_P= 20, print= False ) # C bar 

#Shaft furnace input 
# Define Ammonia input stream properties. Ammonia will be provided to shaft furnace additional to the reformer. 
ammonia_stream = {"name": "Ammonia Stream","T":580,"P":1,"amount": 268, "percentage":{(0,"NH3"): 100}} # C bar kg wt.%
hydrogen_stream = {"name": "Hydrogen Stream","T":580,"P":1,"amount": 0, "percentage":{(0,"H2"): 100}} # C bar kg wt.%
# Define Iron Oxide stream properties  
iron_oxide_stream = {"name": "Iron Oxide Stream","T":25,"P":1,"amount":1000, "percentage":{("Fe2O3_hematite(s)","Fe2O3_hematite(s)"): 100}} #kg

#Run shaft_furnace()
stream_top_gas, stream_DRI, shaft_furnace_enthalpy_change, metallization_degree = shaft_furnace(ammonia_stream,hydrogen_stream,iron_oxide_stream,reducing_gas_stream, 1000, 1, print= True) #C bar

#Run energy_balance()
energy_requirement, recovered_energy_top_gas = energy_balance(stream_top_gas, shaft_furnace_enthalpy_change, reformer_enthalpy_change)

## Results 
Here we can demonstrate the results of the Midrex process. An overview of the stream management and process flowchart for current process model is described below. 
 
![Midrex flowchart ](midrex_chart.png)

### 1. Reformer stream

In [ ]:
print(f"Reformer stream\n {df_reformer}\n")
#The reducing gas ratio (H2:CO) is typically around 1.55(in volume) or 0.11 (in mass) 
reducing_gas_ratio = df_reformer.loc["H2","Gas Output kg"] / df_reformer.loc["CO","Gas Output kg"]
print(f"Reducing gas ratio (in mass): {round(reducing_gas_ratio,2)}")

We can compare the results with the actual plant data:
|  Gas  | Input (wt.%)         | Output (wt.%)        |
|-------|----------------------|----------------------|
| H2    | 3.92                 | 8.33                 |
| CO    | 29.60                | 73.63                |
| CO2   | 36.72                | 6.61                 |
| H2O   | 13.03                | 8.12                 |
| CH4   | 15.17                | 1.20                 |
| N2    | 1.56                 | 2.10                 |

Reducing gas ratio (in mass): 0.11, (in volume): 1.55 

Source: MIDREX NG Brochure (4.12.2018)

### 2. Shaft furnace: top gas 

In [ ]:
print(f"Stream top gas\n {stream_top_gas}\n")


### 3. Shaft furnace: DRI and metallization degree

In [ ]:
print(f"Stream DRI\n {stream_DRI}\n")

In [ ]:
import matplotlib.pyplot as plt

def draw_pie_chart(metallization_degree):
    labels = ['DRI', 'Oxide']
    sizes = [metallization_degree * 100, abs((1 - metallization_degree)) * 100]

    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    plt.title(f'Metallization Degree: {metallization_degree:.2f}')
    plt.show()

print(f"Metallization degree: {metallization_degree}, max 1")
draw_pie_chart(metallization_degree)

### 4. Energy requirement

In [ ]:
print(f"Energy requirement: {round(energy_requirement,2)} {cau.get_E_unit().name}")
print(f"Recovered energy from top gas { recovered_energy_top_gas :.2f} {cau.get_E_unit().name}, "
        f"Shaft furnace energy requirement {shaft_furnace_enthalpy_change:.2f} {cau.get_E_unit().name}, "
        f"Reformer energy requirement {reformer_enthalpy_change:.2f} {cau.get_E_unit().name}")

labels = ['Top Gas', 'Shaft Furnace', 'Reformer', 'Total Required']
values = [recovered_energy_top_gas , shaft_furnace_enthalpy_change, reformer_enthalpy_change, energy_requirement]
plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'], alpha = 0.7)
plt.ylabel(f'Energy Consumption ({cau.get_E_unit().name})')
plt.title('Energy Balance')
plt.show()
